In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/tmdb-movie-metadata/tmdb_5000_credits.csv
/kaggle/input/tmdb-movie-metadata/tmdb_5000_movies.csv


In [2]:
import pandas as pd
tmdb_5000_credits = pd.read_csv("../input/tmdb-movie-metadata/tmdb_5000_credits.csv")
tmdb_5000_movies = pd.read_csv("../input/tmdb-movie-metadata/tmdb_5000_movies.csv")

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
#print the first 4 rows of data
tmdb_5000_movies.head(5)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [5]:
#count the number of rows and columns
tmdb_5000_movies.shape


(4803, 20)

In [6]:
#create a list of the important features of the movie
features = ['keywords','production_companies','genres','vote_average']
#printing the first 4 rows of data in relation to th features of the movie
tmdb_5000_movies[features].head(4)

,keywords,production_companies,genres,vote_average
0,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",7.2
1,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",6.9
2,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",6.3
3,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",7.6


In [7]:
#Clean and preprocess the data
for feature in features:
    #Fills any missing values from the data
    tmdb_5000_movies[feature] = tmdb_5000_movies[feature].fillna('')

In [8]:
#Combine all of the important values in a single string
def combine_features(row):
    return row['keywords']+' '+row['production_companies']+' '+row['genres']+' '+row['spoken_languages']

In [9]:
# Using the function combine_features() we create a new column called combined_feature
tmdb_5000_movies['combined_features'] = tmdb_5000_movies.apply(combine_features, axis=1)

In [10]:
#add an index column to the dataset
tmdb_5000_movies['index1'] = tmdb_5000_movies.index

In [11]:
#show the new column
tmdb_5000_movies.head(2)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,combined_features,index1
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",0
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",...,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",1


In [12]:
#convert a collection of text to a matrix of tocken counts
count_matrix = CountVectorizer().fit_transform(tmdb_5000_movies['combined_features'])

In [13]:
#Get the cosine similarity matrix from the count matrix (cos(theta))
cosine_sim = cosine_similarity(count_matrix)

#Print the cosine similarity matrix
print(cosine_sim)

[[1.         0.95144959 0.91964378 ... 0.93012192 0.40063498 0.9004511 ]
 [0.95144959 1.         0.91482558 ... 0.92775538 0.39668083 0.89816697]
 [0.91964378 0.91482558 1.         ... 0.89707924 0.52634495 0.87701885]
 ...
 [0.93012192 0.92775538 0.89707924 ... 1.         0.42823018 0.88454846]
 [0.40063498 0.39668083 0.52634495 ... 0.42823018 1.         0.47336463]
 [0.9004511  0.89816697 0.87701885 ... 0.88454846 0.47336463 1.        ]]


In [14]:
cosine_sim.shape

(4803, 4803)

In [15]:
#function to get the title form the index

def get_title(index):
    return tmdb_5000_movies[tmdb_5000_movies.index == index]['title'].values[0]

def get_index(title):
    return tmdb_5000_movies[tmdb_5000_movies.title == title]['index1'].values[0]

In [16]:
#the movie that the user chooses
movie_choice = "Avatar"

#find that movies index
movie_index = get_index(movie_choice)

In [17]:
#Access the row, through the movies index, corresponding to this movie (the liked movie) in the similarity matrix, 
# by doing this we will get the similarity scores of all other movies from the current movie

#Enumerate through all the similarity scores of that movie to make a tuple of movie index and similarity scores.
#  This will convert a row of similarity scores like this- [5 0.6 0.3 0.9] to this- [(0, 5) (1, 0.6) (2, 0.3) (3, 0.9)] . 
#  Note this puts each item in the list in this form (movie index, similarity score)
similar_movies =  list(enumerate(cosine_sim[movie_index]))



#Sort the list of similar movies according to the similarity scores in descending order
#Since the most similar movie is itself, we will discard the first element after sorting.
sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)[1:]

In [18]:
#Print the sorted similar movies to the movie the user like
# The tuples are in the form (movie_index, similarity value)
print(sorted_similar_movies)

[(838, 0.9691601175714617), (3158, 0.967660187807473), (215, 0.9665863880667469), (2442, 0.9664680300090772), (322, 0.9660824910423825), (2403, 0.964952151455025), (2217, 0.9646364325932494), (278, 0.963629621215503), (1079, 0.963554899210692), (2912, 0.9628870773445543), (1990, 0.9627667996210116), (507, 0.9627417036386764), (2380, 0.9625614304294866), (95, 0.962518270436957), (261, 0.9624408062972208), (454, 0.9622038449748087), (47, 0.9620920563743722), (3534, 0.9619934464821656), (778, 0.9619912786269473), (770, 0.9618416912626093), (123, 0.9617652512015971), (582, 0.9617082724395389), (2799, 0.9615239476408233), (2879, 0.9611936428209896), (165, 0.961065408729902), (740, 0.9609036479430975), (1213, 0.9606993242925013), (156, 0.9606503094386996), (1433, 0.960619563113671), (2428, 0.960504789409641), (415, 0.960351247990153), (158, 0.9603222671379893), (645, 0.9602865932199373), (1390, 0.9602322582744008), (270, 0.9601735552124163), (1358, 0.9601604316120969), (212, 0.96014326754866

In [19]:
#Create a loop to print the first 10 entries from the sorted similar movies list

i=0
print("Top 10 similar movies to "+movie_choice+" are:")
for element in sorted_similar_movies:
    print(get_title(element[0]) )
    i=i+1
    if i>=10:
        break

Top 10 similar movies to Avatar are:
Alien³
Alien
Fantastic 4: Rise of the Silver Surfer
Southland Tales
The Fifth Element
Aliens
Everyone Says I Love You
Planet of the Apes
Dune
Star Wars
